In [1]:
from skbio import TreeNode
import gzip
from collections import Counter, deque
import time
import numpy as np
import pandas as pd
from bp import parse_newick, from_skbio_treenode
def extract_consenus(file):
    f = open(file)
    g_id = f.readline()[1:].strip()
    seq = ""
    lines = iter(f)
    for line in lines:
        if line[0] == '>':
            yield (g_id, seq)
            g_id = line[1:].strip()
            seq = "" 
        else:
            seq += line.strip()

def extract_sample_read_counts(file):
    f = gzip.open(file, 'rt')
    s_count = Counter()
    for idx, line in enumerate(f):
        if idx % 4 == 1:
            s_count[line.strip()] += 1
    
    return s_count

In [2]:
consenus_genome = deque(
    extract_consenus("2021-08-23_21-01-26-all_stringent_only.fas"))
genomes = [genome for genome, _ in consenus_genome]
sequences = [sequence for _, sequence in consenus_genome]
d = {'genome': genomes, 'sequences': sequences}
consensus_genome_df = pd.DataFrame(data=d)
consensus_genome_df['genome'] = consensus_genome_df.genome.str.replace('_',' ')

# The following is a POC for a single sample

In [3]:
# # sample 38879
r1 = [(count, read) for read, count in extract_sample_read_counts(
        "SEARCH-38879__E0001197__P24__210924_A01535_0019_BHT7MHDSX2__S736_L004_R1_001.fastq.gz").items() 
          if count > 1]
r2 = [(count, seq) for seq, count in extract_sample_read_counts(
        "SEARCH-38879__E0001197__P24__210924_A01535_0019_BHT7MHDSX2__S736_L004_R2_001.fastq.gz").items()
           if count > 1]
tree = TreeNode.read("2021-08-23_21-01-26-all_stringent_refs_hist.trimmed.aln.rooted.treefile",)
for i, n in enumerate(tree.postorder(include_self=True), 1):
    if n.name is None:
        n.name = str(i)
tip_to_postorder = {n.name: i for i, n in enumerate(tree.postorder(include_self=True), 1) if n.is_tip() }
bp_tree = from_skbio_treenode(tree)

## get genomes per amplicon/feature table data

In [4]:
t1 = time.process_time()
observed = {} # keeps track of which amplicons we have seen 
tips_per_read = [] # stores which genomes each unique amplicon is found in
f_table_staged = []

# TODO: will need to iterate over all samples
for count, read in r1 + r2: # TODO: will need to create function to get the r1 and r2 reads per sample
    if count > 10000: # just to test, the lower the value => more amplicons => much slower time
        f_table_staged.append(("sample_id_38879", read, count)) # add sample/amplicon to feature table
        
        if read in observed: # amplicon was already seen in another sample
            continue
        
        found = consensus_genome_df.loc[
            consensus_genome_df.sequences.str.contains(read)
        ]
        
        # currently discarding amplicon if its not found in atleast 1 genome
        # Should we keep it??? 
        if found.size > 0:
            tips_per_read.append((read, [tip_to_postorder[genome] for genome in found.genome])) 
t2 = time.process_time()
print(t2-t1)

13.947154103999999


## insert amplicons into tree via lca of the genomes they were found in

In [6]:
t1 = time.process_time()
def get_lca(tips, bp_tree):
    if len(tips) == 1:
        return bp_tree.name(bp_tree.postorderselect(tips[0]))
    
    lca = bp_tree.postorderselect(tips[0])
    for tip in tips[1:]:
        tip = bp_tree.postorderselect(tip)
        lca = bp_tree.lca(lca, tip)
        
    return bp_tree.name(lca)

for read, tips in tips_per_read:  
    lca = tree.find(get_lca(tips, bp_tree))
    lca.append(TreeNode(name=read))
t2 = time.process_time()
print(t2-t1)

0.020300194999997245
